In [2]:
library(tidyverse)
library(rstan)
library(foreign)
library(here)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
Warning message:
“package ‘ggplot2’ was built under R version 3.4.4”Warning message:
“package ‘tibble’ was built under R version 3.4.3”Warning message:
“package ‘tidyr’ was built under R version 3.4.4”Warning message:
“package ‘purrr’ was built under R version 3.4.4”Warning message:
“package ‘dplyr’ was built under R version 3.4.4”Warning message:
“package ‘stringr’ was built under R version 3.4.3”Warning message:
“package ‘forcats’ was built under R version 3.4.3”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“package ‘rstan’ was built under R version 3.4.3”Loading required package: StanHeaders
Warning message:
“package ‘StanHeaders’ was 

In [3]:
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())

In [4]:
mag_code = "
        data {
        int<lower=0> J; // number of studies
        int<lower=0> Y_m[J];
        int<lower=0> Y_c[J]; // Deaths in both groups
        int<lower=0> N_m[J];
        int<lower=0> N_c[J]; //Total subjects in both groups
        real<lower=0> eta; // Prior sd of mu
        real<lower=0> s; //scaling parameter for prior for tau
        }

        parameters {
            real delta[J]; // log odds ratios
            real<lower=0,upper=1> q[J];  // baseline probability
            real<lower=0> tau; // standard deviation of random effets
            real mu; // 
        }

        transformed parameters {
            real<lower=0,upper=1> p[J];
            real phi[J]; 
            for (j in 1:J){
                phi[j] = logit(q[j]);  // Invert Binomial probs for control
                p[j] = inv_logit(delta[j] + phi[j]); // Binomial probs for response
            }
        }

        model {
            Y_m ~ binomial(N_m,p);
            Y_c ~ binomial(N_c,q);
            delta ~ normal(mu,tau);
            q ~ uniform(0,1);
            tau ~ cauchy(0, s);
            mu ~ normal(0,eta);
        }

        generated quantities { 
        real medianOR; 
        medianOR = exp(mu);
        }
        "

In [5]:
magnes_data=read.dta(here("data/magnes.dta"))
magnes_data_noSch1 = magnes_data %>% filter(trialnam!="Schechter 1") %>% 
  mutate(Study=paste(trialnam,year," ")) %>%
  rename(Year=year,N_m = tot1,N_c=tot0,Y_m=dead1,Y_c=dead0) %>% 
  mutate(p_m=Y_m/N_m,p_c=Y_c/N_c,Beneficial=factor(ifelse(p_m>=p_c,"Harmful","Beneficial")),
         #TotalSize = cut(x=N_m+N_c,breaks=c(0,50,250,1000,2500,5000,25000,60000)))
          TotalSize=N_m+N_c)
#magnes_data_noSch1  %>%  select(Study,N_m:Y_c)%>%kable(.)


Warning message:
“package ‘bindrcpp’ was built under R version 3.4.4”

In [6]:
data_list = with(magnes_data_noSch1, 
                 list(J=nrow(magnes_data_noSch1), 
                      Y_m = Y_m,
                      Y_c = Y_c,
                      N_m = N_m,
                      N_c = N_c,
                      s = 1,
                      eta=NULL)
                     )

In [89]:
data_list[["eta"]] = 0.7
stan_samples_0.7 <- stan(model_code = mag_code, data = data_list,verbose=T)



TRANSLATING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' FROM Stan CODE TO C++ CODE NOW.
successful in parsing the Stan model 'be8ceb788a44a6e7bcdeb561d91170b7'.

CHECKING DATA AND PREPROCESSING FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

COMPILING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

STARTING SAMPLER FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.


In [59]:
print(stan_samples_0.7,pars=c("tau","mu","medianOR"))

Inference for Stan model: be8ceb788a44a6e7bcdeb561d91170b7.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

          mean se_mean   sd  2.5%   25%   50%   75% 97.5% n_eff Rhat
tau       0.64    0.01 0.21  0.31  0.49  0.60  0.75  1.15  1398    1
mu       -0.80    0.00 0.23 -1.27 -0.95 -0.79 -0.64 -0.36  2439    1
medianOR  0.46    0.00 0.11  0.28  0.39  0.45  0.53  0.70  2463    1

Samples were drawn using NUTS(diag_e) at Wed Nov 28 15:57:30 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


In [62]:
data_list[["eta"]] = 0.4
stan_samples_0.4 <- stan(model_code = mag_code, data = data_list,verbose=T)





TRANSLATING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' FROM Stan CODE TO C++ CODE NOW.
successful in parsing the Stan model 'be8ceb788a44a6e7bcdeb561d91170b7'.

CHECKING DATA AND PREPROCESSING FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

COMPILING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

STARTING SAMPLER FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.


Warning message:
“There were 1 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”Warning message:
“Examine the pairs() plot to diagnose sampling problems
”

In [63]:
print(stan_samples_0.4,pars=c("tau","mu","medianOR"))

Inference for Stan model: be8ceb788a44a6e7bcdeb561d91170b7.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

          mean se_mean   sd  2.5%   25%   50%   75% 97.5% n_eff Rhat
tau       0.60    0.01 0.22  0.27  0.45  0.57  0.71  1.10  1463    1
mu       -0.66    0.00 0.20 -1.05 -0.79 -0.65 -0.52 -0.27  3252    1
medianOR  0.53    0.00 0.11  0.35  0.46  0.52  0.59  0.77  3188    1

Samples were drawn using NUTS(diag_e) at Wed Nov 28 16:13:11 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


In [64]:
data_list[["eta"]] = 0.1
stan_samples_0.1 <- stan(model_code = mag_code, data = data_list,verbose=T)




TRANSLATING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' FROM Stan CODE TO C++ CODE NOW.
successful in parsing the Stan model 'be8ceb788a44a6e7bcdeb561d91170b7'.

CHECKING DATA AND PREPROCESSING FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

COMPILING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

STARTING SAMPLER FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.


In [65]:
print(stan_samples_0.1,pars=c("tau","mu","medianOR"))

Inference for Stan model: be8ceb788a44a6e7bcdeb561d91170b7.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

          mean se_mean   sd  2.5%   25%   50%   75% 97.5% n_eff Rhat
tau       0.89    0.01 0.34  0.32  0.65  0.86  1.10  1.66  1368    1
mu       -0.11    0.00 0.10 -0.31 -0.19 -0.11 -0.04  0.08  4000    1
medianOR  0.90    0.00 0.09  0.73  0.83  0.89  0.96  1.09  4000    1

Samples were drawn using NUTS(diag_e) at Wed Nov 28 16:14:07 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


In [68]:
print(eval(as.name(paste("stan_samples_",0.1,sep=""))),pars=c("tau","mu","medianOR"))

Inference for Stan model: be8ceb788a44a6e7bcdeb561d91170b7.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

          mean se_mean   sd  2.5%   25%   50%   75% 97.5% n_eff Rhat
tau       0.89    0.01 0.34  0.32  0.65  0.86  1.10  1.66  1368    1
mu       -0.11    0.00 0.10 -0.31 -0.19 -0.11 -0.04  0.08  4000    1
medianOR  0.90    0.00 0.09  0.73  0.83  0.89  0.96  1.09  4000    1

Samples were drawn using NUTS(diag_e) at Wed Nov 28 16:14:07 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


In [20]:
one_stan_out = function(x){ 
    data_frame(medianOR=rstan::extract(eval(as.name(paste("stan_samples_",x,sep=""))),
                                       par="medianOR")$medianOR, 
               eta = x)
               
}

In [21]:
my_output_df = c("0.1","0.4","0.7") %>% map_df(~one_stan_out(.))

ERROR: Error in eval(as.name(paste("stan_samples_", x, sep = ""))): object 'stan_samples_0.1' not found


In [94]:
write_csv(my_output_df,here("results/stan_out_147.csv"))

In [7]:
data_list[["eta"]] = 1.0
stan_samples_1.0 <- stan(model_code = mag_code, data = data_list,verbose=T)




TRANSLATING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' FROM Stan CODE TO C++ CODE NOW.
successful in parsing the Stan model 'be8ceb788a44a6e7bcdeb561d91170b7'.
OS: x86_64, darwin15.6.0; rstan: 2.17.3; Rcpp: 0.12.18; inline: 0.3.14 
 >> setting environment variables: 
PKG_LIBS =  -L'/Library/Frameworks/R.framework/Versions/3.4/Resources/library/StanHeaders/lib' -lStanHeaders
PKG_CPPFLAGS =   -I"/Library/Frameworks/R.framework/Versions/3.4/Resources/library/Rcpp/include/"  -I"/Library/Frameworks/R.framework/Versions/3.4/Resources/library/RcppEigen/include/"  -I"/Library/Frameworks/R.framework/Versions/3.4/Resources/library/RcppEigen/include/unsupported"  -I"/Library/Frameworks/R.framework/Versions/3.4/Resources/library/rstan/include/boost_not_in_BH" -I"/Library/Frameworks/R.framework/Versions/3.4/Resources/library/BH/include" -I"/Library/Frameworks/R.framework/Versions/3.4/Resources/library/StanHeaders/include/src/"  -I"/Library/Frameworks/R.framework/Versions/3.4/Resources/library/StanHe

In [16]:
data_frame(v1=rstan::extract(stan_samples_1.0,par="medianOR")$medianOR) %>% 
count(v1<1) %>% mutate(prop=n/sum(n))

v1 < 1,n,prop
FALSE,2,0.0005
TRUE,3998,0.9995


In [23]:
my_output_df_one = c("1.0") %>% map_df(~one_stan_out(.))
write_csv(my_output_df_one,here("results/stan_out_1.0.csv"))

In [24]:
data_list[["eta"]] = 0.001
stan_samples_0.001 <- stan(model_code = mag_code, data = data_list,verbose=T)




TRANSLATING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' FROM Stan CODE TO C++ CODE NOW.
successful in parsing the Stan model 'be8ceb788a44a6e7bcdeb561d91170b7'.

CHECKING DATA AND PREPROCESSING FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

COMPILING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

STARTING SAMPLER FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.


In [25]:
data_frame(v1=rstan::extract(stan_samples_0.001,par="medianOR")$medianOR) %>% 
count(v1<1) %>% mutate(prop=n/sum(n))

v1 < 1,n,prop
FALSE,1978,0.4945
TRUE,2022,0.5055


In [26]:
my_output_df_0.001 = c("0.001") %>% map_df(~one_stan_out(.))
write_csv(my_output_df_0.001,here("results/stan_out_001.csv"))

In [27]:
data_list[["eta"]] = 0.55
stan_samples_0.55 <- stan(model_code = mag_code, data = data_list,verbose=T)





TRANSLATING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' FROM Stan CODE TO C++ CODE NOW.
successful in parsing the Stan model 'be8ceb788a44a6e7bcdeb561d91170b7'.

CHECKING DATA AND PREPROCESSING FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

COMPILING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

STARTING SAMPLER FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.


In [28]:
data_frame(v1=rstan::extract(stan_samples_0.55,par="medianOR")$medianOR) %>% 
count(v1<1) %>% mutate(prop=n/sum(n))

v1 < 1,n,prop
FALSE,2,0.0005
TRUE,3998,0.9995


In [35]:
my_output_df_0.55 = c("0.55") %>% map_df(~one_stan_out(.))
write_csv(my_output_df_0.55,here("results/stan_out_55.csv"))

In [30]:
data_list[["eta"]] = 0.86
stan_samples_0.86 <- stan(model_code = mag_code, data = data_list,verbose=T)





TRANSLATING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' FROM Stan CODE TO C++ CODE NOW.
successful in parsing the Stan model 'be8ceb788a44a6e7bcdeb561d91170b7'.

CHECKING DATA AND PREPROCESSING FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

COMPILING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

STARTING SAMPLER FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.


In [32]:
data_frame(v1=rstan::extract(stan_samples_0.86,par="medianOR")$medianOR) %>% 
count(v1<1) %>% mutate(prop=n/sum(n))

v1 < 1,n,prop
FALSE,2,0.0005
TRUE,3998,0.9995


In [36]:
my_output_df_0.86 = c("0.86") %>% map_df(~one_stan_out(.))
write_csv(my_output_df_0.86,here("results/stan_out_86.csv"))

In [40]:
data_list[["eta"]] = 0.11
stan_samples_0.11 <- stan(model_code = mag_code, data = data_list,verbose=T)




TRANSLATING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' FROM Stan CODE TO C++ CODE NOW.
successful in parsing the Stan model 'be8ceb788a44a6e7bcdeb561d91170b7'.

CHECKING DATA AND PREPROCESSING FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

COMPILING MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.

STARTING SAMPLER FOR MODEL 'be8ceb788a44a6e7bcdeb561d91170b7' NOW.


In [41]:
data_frame(v1=rstan::extract(stan_samples_0.11,par="medianOR")$medianOR) %>% 
count(v1<1) %>% mutate(prop=n/sum(n))

v1 < 1,n,prop
FALSE,389,0.09725
TRUE,3611,0.90275


In [42]:
my_output_df_0.11 = c("0.11") %>% map_df(~one_stan_out(.))
write_csv(my_output_df_0.11,here("results/stan_out_11.csv"))

In [ ]:
### Needed to run twice due to bad warmup on seed
data_list[["eta"]] = 0.11
stan_samples_0.11 <- stan(model_code = mag_code, data = data_list,verbose=T)
data_frame(v1=rstan::extract(stan_samples_0.11,par="medianOR")$medianOR) %>% 
count(v1<1) %>% mutate(prop=n/sum(n))
my_output_df_0.11 = c("0.11") %>% map_df(~one_stan_out(.))
write_csv(my_output_df_0.11,here("results/stan_out_11.csv"))
